Installing Haystack


In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]

In [27]:
from haystack.telemetry import tutorial_running

tutorial_running(1)

In [28]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [29]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [31]:
doc_dir = "Data"

In [ ]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)


In [33]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [34]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [35]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
pip install gradio

In [37]:
import gradio as gr

In [ ]:
# prediction = pipe.run(
#     query="What are some example of cardiovascular exercises?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
# )

In [ ]:
# from pprint import pprint

# pprint(prediction)

In [ ]:
# from haystack.utils import print_answers

# print_answers(prediction, details="all")  ## Choose from `minimum`, `medium`, and `all`

In [38]:
def get_highest_score_answer(prediction):
    answers = prediction['answers']
    highest_score = max(answer.score for answer in answers)
    highest_score_answers = [answer for answer in answers if answer.score == highest_score]
    return highest_score_answers


In [ ]:
# highest_score_answers = get_highest_score_answer(prediction)
# for answer in highest_score_answers:
#     print(answer.answer)


In [39]:
def answer_question(query):
    prediction = pipe.run(query=query, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})
    highest_score_answers = get_highest_score_answer(prediction)
    return [answer.answer for answer in highest_score_answers]


In [40]:
iface = gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs="text",
    title="Question Answering Chatbot",
    description="Enter a question and get the answer from the pre-trained model.",
)

In [41]:
iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>